In [1]:
import pandas as pd

In [2]:
name_df = pd.read_csv("name_data.csv",sep="\t")

In [3]:
name_df.shape

(170002, 2)

In [4]:
profane_df = pd.read_csv("profane_words.csv",sep="\t")

In [5]:
profane_df.shape

(169988, 2)

In [6]:
final_df = name_df.append(profane_df, ignore_index=True)

In [7]:
final_df=final_df.sample(frac=1).reset_index(drop=True)

In [8]:
final_df.isna().sum()

TEXT     2
LABEL    0
dtype: int64

In [ ]:
final_df.head(n=10)

,TEXT,LABEL
0,hera,NOTNAME
1,Durgesh Ballav Raisaili,NAME
2,Kulananda Ballav Sharama,NAME
3,hunch,NOTNAME
4,Yubika Asma,NAME
5,okay shit whrted,NOTNAME
6,paudaina,NOTNAME
7,Jyotika Rana Buda,NAME
8,afno,NOTNAME
9,Mohanish Prasad Eshwar,NAME


In [ ]:
for l in final_df["TEXT"]:
    if type(l) != str:
        print("bad")
        print(l)

bad
nan
bad
nan


In [ ]:
final_df.loc[:,"TEXT"] = final_df.TEXT.apply(lambda x : str(x))

Data splitting

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
vocabulary_size = 20000
tokenizer = Tokenizer(char_level=False, oov_token='UNK',num_words = vocabulary_size)
tokenizer.fit_on_texts(final_df["TEXT"])
sequences = tokenizer.texts_to_sequences(final_df['TEXT'])
data = pad_sequences(sequences,maxlen = 10)

Using TensorFlow backend.


In [ ]:
tokenizer.index_word

{1: 'UNK',
 2: 'rana',
 3: 'chandra',
 4: 'kumar',
 5: 'nath',
 6: 'ballav',
 7: 'jung',
 8: 'bahadur',
 9: 'prasad',
 10: 'aditya',
 11: 'lol',
 12: 'september',
 13: 'com',
 14: 'haha',
 15: 'tyo',
 16: 'october',
 17: 'nai',
 18: 'bholi',
 19: 'hai',
 20: 'haru',
 21: 'hola',
 22: 'pani',
 23: 'kati',
 24: 'malai',
 25: 'haina',
 26: 'lala',
 27: 'timi',
 28: 'tero',
 29: 'xaina',
 30: 'okay',
 31: 'aja',
 32: 'bhayo',
 33: 'gardai',
 34: 'maile',
 35: 'chha',
 36: 'shubham',
 37: 'bhane',
 38: 'aah',
 39: 'na',
 40: 'jana',
 41: 'pratik',
 42: 'hunxa',
 43: 'bhan',
 44: 'bata',
 45: 'kasto',
 46: 'colz',
 47: 'chaina',
 48: 'andresh',
 49: 'garna',
 50: 'garne',
 51: 'timro',
 52: 'kaile',
 53: 'ghar',
 54: 'co',
 55: 'omar',
 56: 'huncha',
 57: 'tei',
 58: 'bhai',
 59: 'k',
 60: 'samma',
 61: 'lo',
 62: 'bhaneko',
 63: 'kei',
 64: 'sha',
 65: 'kun',
 66: 'ahh',
 67: 'sahil',
 68: 'www',
 69: 'wala',
 70: 'dherai',
 71: 'jasto',
 72: 'baje',
 73: 'xa',
 74: 'hahaha',
 75: 'chhaina'

In [ ]:
final_df["LABEL"] = final_df["LABEL"].replace(["NAME" , "NOTNAME"] , [1 , 0])

In [ ]:
labels = final_df["LABEL"]

In [ ]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense,  LSTM, Conv1D, MaxPooling1D, Dropout, Activation
import numpy as np
from keras.layers import BatchNormalization

In [ ]:
def create_model(optimizer='adam'):
    model2 = Sequential()
    model2.add(Embedding(20000, 50, input_length=10))
    model2.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
    model2.add(BatchNormalization())
    model2.add(Dropout(0.4))
    model2.add(Dense(50, activation="tanh"))
    model2.add(BatchNormalization())
    model2.add(Dropout(0.4))
    model2.add(Dense(50, activation="tanh"))
    model2.add(BatchNormalization())
    model2.add(Dropout(0.4))
    model2.add(Dense(1, activation="sigmoid"))
    model2.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model2

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn = create_model)

In [ ]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(data, np.array(labels))

/home/ai5/anaconda3/envs/infodevai/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
